# LSTM Implementation in TreNet

### Authors: Nathan Ng, Gao Mo, Richard Tang

### LSTM Implementation

- LSTM model that feeds into linear layer that matches number of outputs as CNN stack
- Takes as input stock trend durations and slopes 

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Use GPU if available for PyTorch
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = torch.device(dev)

## Data Pre-Processing

In [3]:
"""
Scales data using Sklearn's MinMaxScaler
Generalized to accept tensors 
"""
class Scaler():
    def __init__(self):
        self.scaler = MinMaxScaler()
        
    def fit_transform(self, data):
        # Check if data is tensor
        if type(data) == torch.Tensor:
            data = torch.Tensor.cpu(data).detach().numpy()
            
        # Check if data is dataframe 
        if type(data) == pd.DataFrame or type(data) == pd.Series:
            data = data.values
            
        # Transform data 
        if len(data.shape) == 1: 
            scaled_data = self.scaler.fit_transform(data.reshape(-1, 1)).flatten()
        else: 
            scaled_data = self.scaler.fit_transform(data)
        
        # Return tensor of scaled data
        return torch.tensor(scaled_data, dtype=torch.float)
    
    def inverse_transform(self, data): 
        # Check if data is tensor
        if type(data) == torch.Tensor:
            data = torch.Tensor.cpu(data).detach().numpy()
            
        # Check if data is dataframe 
        if type(data) == pd.DataFrame or type(data) == pd.Series:
            data = data.values
        
        inverse_data = self.scaler.inverse_transform(data)
        
        # Return tensor of inverse data
        return torch.tensor(inverse_data, dtype=torch.float)

In [4]:
"""
Extracts m sequential data to use to predict n next data 
"""
def extract_data(data, num_input, num_output):
    num_rows = data.shape[0] - num_input - num_output
    input_data = torch.zeros(num_rows, num_input)
    output_data = torch.zeros(num_rows, num_output)
    
    for i in range(num_rows):
        input_data[i] = (data[i:i+num_input])
        output_data[i] = (data[i+num_input:i+num_input+num_output])
    return input_data, output_data

In [5]:
"""
Separates data into train, validation, and test sets
props: (train, valid)
"""
def train_valid_test_split(X, y, props=None):
    if not props: 
        props = (0.5, 0.25)
    elif len(props) != 3: 
        print("Wrong number of parameters")
        return None
    
    train_size = int(X.shape[0] * props[0])
    valid_size = int(X.shape[0] * props[1])
    
    X_train = X[:train_size].to(device)
    y_train = y[:train_size].to(device)
    
    X_valid = X[train_size: (train_size + valid_size)].to(device)
    y_valid = y[train_size: (train_size + valid_size)].to(device)
    
    X_test = X[(train_size + valid_size):].to(device)
    y_test = y[(train_size + valid_size):].to(device)
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

## TreNet LSTM Model

In [6]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        
        # Initialize hidden dimenision and layers 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.input_dim = input_dim
        
        # Initialize deep learning models
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        
        # Reshape data if needed
        if len(x.shape) != 3: 
            x = x.reshape(x.shape[0], -1, self.input_dim)
        
        # Run data through model
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

## TreNet LSTM Training

In [7]:
"""
Create training loop
"""
def train_loop(n_epochs, X, y, model, loss_fn, optimizer, printout=False, record_loss=False):
    losses = []
    for i in range(n_epochs):
        # Compute prediction and loss 
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print loss per epoch 
        if printout and i % 100 == 0: 
            print(f"Epoch {i}:\n--------------")
            print(f"Train Loss: {np.sqrt(loss.item())}")
            print()
            
        # Record loss per epoch 
        if record_loss: 
            losses.append(np.sqrt(loss.item()))
            
    # Print final loss after training 
    if printout:
        print(f"Final:\n--------------")
        print(f"Train Loss: {np.sqrt(loss.item())}")
        print()
    
    if record_loss:
        return losses

## Test LSTM Model on Raw Stock Prices

In [8]:
# Load in NYSE data
df = pd.read_csv("../data/raw/indexProcessed.csv")
NYSE_df = df[df["Index"] == "NYA"].loc[:, "Open"].reset_index(drop=True)

# Create subset of data 
NYSE_sub = NYSE_df[:1000]

In [9]:
# Scale data 
scaler = Scaler()
sub = scaler.fit_transform(NYSE_sub).to(device)

In [10]:
# Extract samples and create train test sets
X, y = extract_data(sub, 49, 1)
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(X, y)

In [11]:
# Set parameters for model and training 
input_dim = 1
hidden_dim = 32
num_layers = 1
output_dim = 1

learning_rate = 0.01
num_epochs = 4000

In [12]:
# Initialize model, loss function, and optimizer
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Train model
training_loss = train_loop(num_epochs, X_train, y_train, model, loss_fn, optimizer, printout=True, record_loss=True)

Epoch 0:
--------------
Train Loss: 0.419464358887889

Epoch 100:
--------------
Train Loss: 0.02287488169375688

Epoch 200:
--------------
Train Loss: 0.020361609079631825

Epoch 300:
--------------
Train Loss: 0.018383628912908262

Epoch 400:
--------------
Train Loss: 0.017281342377952518

Epoch 500:
--------------
Train Loss: 0.016709302414833118

Epoch 600:
--------------
Train Loss: 0.016208899611184685

Epoch 700:
--------------
Train Loss: 0.015729714410163825

Epoch 800:
--------------
Train Loss: 0.015262119883346973

Epoch 900:
--------------
Train Loss: 0.014988692651674459

Epoch 1000:
--------------
Train Loss: 0.014539750029214605

Epoch 1100:
--------------
Train Loss: 0.014176732054467112

Epoch 1200:
--------------
Train Loss: 0.025136093659472225

Epoch 1300:
--------------
Train Loss: 0.01383539003763757

Epoch 1400:
--------------
Train Loss: 0.013592209595961732

Epoch 1500:
--------------
Train Loss: 0.013429543525023633

Epoch 1600:
--------------
Train Loss: 0.

In [14]:
# Compare predictions and actual values 
inverse_test_y = scaler.inverse_transform(y_test)

pred_test_y = model(X_test)
inverse_pred_test_y = scaler.inverse_transform(pred_test_y)

In [15]:
print("Test Loss: " + str(loss_fn(inverse_pred_test_y, inverse_test_y)**(1/2)))

print("Predicted values: \n" + str(inverse_test_y[:10]))

print("Actual values: \n" + str(inverse_pred_test_y[:10]))

Test Loss: tensor(3.8764)
Predicted values: 
tensor([[605.6600],
        [600.9100],
        [595.4100],
        [587.7900],
        [583.5600],
        [586.2100],
        [583.7800],
        [583.5600],
        [584.5200],
        [590.2300]])
Actual values: 
tensor([[606.8343],
        [600.8229],
        [596.2324],
        [590.3943],
        [582.6860],
        [579.3992],
        [583.0239],
        [579.6013],
        [580.7210],
        [582.1647]])


In [16]:
# Save training loss as csv 
pd.DataFrame(training_loss).to_csv("../data/losses/lstm_loss.csv", index=False)